In [25]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,AveragePooling2D,Flatten
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint


In [26]:
(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()

X_train,X_test=X_train/255,X_test/255
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)

In [27]:
aug_data=ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.3)
aug_data.fit(X_train)

In [28]:
shape=(32,32,3)

inputs = keras.Input(shape=shape)

a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(inputs)  #1
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)   #2
a_temp=a
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)#\3
a=layers.Conv2D(32,(3,3),strides=1,padding="same")(a)#4
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #1
                               
#a_temp=a
##########################################################################
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #5
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #6
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #7
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #8
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #2

###############################################################################

a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #9
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #10
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #11
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #12
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #3

#################################################################################

a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #13
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #14
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #15
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #16
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
############################################################3
a=layers.AveragePooling2D(pool_size=(2,2),strides=1,padding="same")(a)

a=Flatten()(a)

#############################################################

a=layers.Dense(512,activation="relu")(a)
outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)


model = keras.Model(inputs, outputs)

In [29]:
sgd = SGD(lr=0.01, momentum=0.91,clipvalue=0.6)


model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=["accuracy"])
call=EarlyStopping(monitor="val_loss",verbose=1,mode="auto",patience=5,restore_best_weights=True)
checkpoint=ModelCheckpoint('RESNET_SGD_WITHOUT_REGULAIZATION.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True,save_weights_only=True,model='auto',period=1)


In [30]:
model.fit_generator(aug_data.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)/128,epochs=80,validation_data=(X_test,y_test),verbose=1,callbacks=[call,checkpoint])

Epoch 1/80
391/390 [==============================] - ETA: 0s - loss: 4.3422 - accuracy: 0.0426
Epoch 00001: val_accuracy improved from -inf to 0.09400, saving model to sgd_non_reg_VGG16_weights.hdf5
391/390 [==============================] - 84s 214ms/step - loss: 4.3422 - accuracy: 0.0426 - val_loss: 3.9662 - val_accuracy: 0.0940
Epoch 2/80
391/390 [==============================] - ETA: 0s - loss: 3.8266 - accuracy: 0.1117
Epoch 00002: val_accuracy improved from 0.09400 to 0.16640, saving model to sgd_non_reg_VGG16_weights.hdf5
391/390 [==============================] - 83s 213ms/step - loss: 3.8266 - accuracy: 0.1117 - val_loss: 3.5412 - val_accuracy: 0.1664
Epoch 3/80
391/390 [==============================] - ETA: 0s - loss: 3.3938 - accuracy: 0.1902
Epoch 00003: val_accuracy improved from 0.16640 to 0.24970, saving model to sgd_non_reg_VGG16_weights.hdf5
391/390 [==============================] - 83s 213ms/step - loss: 3.3938 - accuracy: 0.1902 - val_loss: 3.0437 - val_accuracy:

In [33]:
y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

In [34]:
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print(accuracy_score(Y_test,y_pred))
print(precision_score(Y_test,y_pred,average="weighted"))
print(recall_score(Y_test,y_pred,average="weighted"))

              precision    recall  f1-score   support

           0       0.86      0.77      0.81       100
           1       0.70      0.73      0.72       100
           2       0.53      0.46      0.49       100
           3       0.35      0.40      0.38       100
           4       0.51      0.31      0.39       100
           5       0.60      0.50      0.55       100
           6       0.68      0.69      0.69       100
           7       0.52      0.64      0.58       100
           8       0.57      0.79      0.66       100
           9       0.66      0.67      0.67       100
          10       0.42      0.37      0.39       100
          11       0.40      0.42      0.41       100
          12       0.50      0.74      0.59       100
          13       0.50      0.63      0.56       100
          14       0.38      0.63      0.48       100
          15       0.50      0.61      0.55       100
          16       0.68      0.64      0.66       100
          17       0.64    

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,AveragePooling2D,Flatten
import tensorflow.keras as keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()

X_train,X_test=X_train/255,X_test/255
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)

shape=(32,32,3)

inputs = keras.Input(shape=shape)

a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(inputs)  #1
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)   #2
a_temp=a
a=layers.Conv2D(32,(3,3),strides=1,padding="same",activation="relu")(a)#\3
a=layers.Conv2D(32,(3,3),strides=1,padding="same")(a)#4
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #1
                               
#a_temp=a
##########################################################################
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #5
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #6
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(64,(3,3),strides=1,padding="same",activation="relu")(a) #7
a=layers.Conv2D(64,(3,3),strides=1,padding="same")(a) #8
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #2

###############################################################################

a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #9
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #10
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(128,(3,3),strides=1,padding="same",activation="relu")(a) #11
a=layers.Conv2D(128,(3,3),strides=1,padding="same")(a) #12
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)

b=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #3

#################################################################################

a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #13
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #14
a=layers.Add()([b,a])
a=layers.Activation("relu")(a)
a_temp=a
a=layers.Conv2D(256,(3,3),strides=1,padding="same",activation="relu")(a) #15
a=layers.Conv2D(256,(3,3),strides=1,padding="same")(a) #16
a=layers.Add()([a,a_temp])
a=layers.Activation("relu")(a)
############################################################3
a=layers.AveragePooling2D(pool_size=(2,2),strides=1,padding="same")(a)

a=Flatten()(a)

#############################################################

a=layers.Dense(512,activation="relu")(a)
outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)


model = keras.Model(inputs, outputs)

model.load_weights('../weights/RESNET_SGD_WITHOUT_REGULAIZATION.hdf5')


y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("the accuracy obtained is {}".format(accuracy_score(Y_test,y_pred)))
print("precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("the recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))

              precision    recall  f1-score   support

           0       0.76      0.79      0.77       100
           1       0.82      0.63      0.71       100
           2       0.46      0.40      0.43       100
           3       0.53      0.16      0.25       100
           4       0.51      0.18      0.27       100
           5       0.52      0.57      0.55       100
           6       0.55      0.73      0.63       100
           7       0.61      0.57      0.59       100
           8       0.57      0.80      0.67       100
           9       0.84      0.61      0.71       100
          10       0.46      0.31      0.37       100
          11       0.46      0.32      0.38       100
          12       0.62      0.66      0.64       100
          13       0.49      0.62      0.55       100
          14       0.53      0.59      0.56       100
          15       0.47      0.60      0.53       100
          16       0.49      0.76      0.60       100
          17       0.75    